## TEST NEW

In [1]:
# lazy import
import sys
sys.path.append("/Users/rachelcohen/hogtie/hogtie")

In [2]:
import hogtie
hogtie.set_loglevel("INFO")

In [3]:
import subprocess
import toytree
import numpy as np

### test data

In [4]:
TREE = toytree.rtree.baltree(ntips=10, treeheight=10)
DATA = np.array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1]).astype(float)

TREE.draw()

(<toyplot.canvas.Canvas at 0x7fd74240cb20>,
 <toytree.Render.ToytreeMark at 0x7fd742417a00>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te967f7d451414d5cb3a8de938e17f69a"> r0 r1 r2 r3 r4 r5 r6 r7 r8 r9

### Fit with hogtie

In [5]:
model = hogtie.BinaryStateModel(TREE, DATA)
model.optimize()

/Users/rachelcohen/hogtie/hogtie/hogtie/binary_state_model.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  qmat = np.array([


ValueError: setting an array element with a sequence.

In [6]:
# inferred parameters
model.alpha, model.beta

(array([0.1       , 0.10000001]), 0.1)

In [8]:
# negative log-likelihood of the data given the model
model.log_lik

6.3

In [9]:
# draw inferred states
model.draw_states();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="300.0px" viewBox="0 0 400.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tcc02529f3c72451d9e277f383605ffb6"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9

In [10]:
# show relative likelihoods
model.tree.get_node_values("likelihood", 1, 1)

array([[0.00184102, 0.00184102],
       [0.02144592, 0.07002355],
       [0.07002349, 0.02144588],
       [0.20487375, 0.20487364],
       [0.04134843, 0.40511919],
       [0.20487375, 0.20487364],
       [0.4051188 , 0.04134834],
       [0.03030872, 0.68212097],
       [0.6821207 , 0.03030867],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

### TEST WITH R

In [11]:

out = subprocess.run([
    "Rscript", 
    "-e", "library(ape)",
    "-e", f"tree <- read.tree(text='{TREE.newick}')",
    "-e", f"data <- c{tuple(DATA[::-1])}",
    "-e", "object <- ace(data, tree, marginal=FALSE, type='discrete', model='ARD', use.expm=FALSE, use.eigen=TRUE)",
    "-e", "print(object)",
    "-e", "print(object$lik.anc)",
    ], capture_output=True,
)

print(out.stdout.decode())


    Ancestral Character Estimation

Call: ace(x = data, phy = tree, type = "discrete", model = "ARD", use.expm = FALSE, 
    use.eigen = TRUE, marginal = FALSE)

    Log-likelihood: -5.604288 

Rate index matrix:
  0 1
0 . 2
1 1 .

Parameter estimates:
 rate index estimate std-err
          1   0.0856  0.0579
          2   0.0856  0.0579

Scaled likelihoods at the root (type '...$lik.anc' to get them for all nodes):
        0         1 
0.4999999 0.5000001 
               0          1
 [1,] 0.49999989 0.50000011
 [2,] 0.32649275 0.67350725
 [3,] 0.38691001 0.61308999
 [4,] 0.12070829 0.87929171
 [5,] 0.02917716 0.97082284
 [6,] 0.67350705 0.32649295
 [7,] 0.61308982 0.38691018
 [8,] 0.87929160 0.12070840
 [9,] 0.97082281 0.02917719



In [13]:
(0.00184102*0.5)+(0.00184102*0.5)

0.00184102

In [15]:
np.log(0.00184102)

-6.297435513129152

In [4]:
TREE1=toytree.rtree.unittree(ntips = 4)
DATA1=np.array([1, 1, 0, 0]).astype(float)

TREE1.draw()
TREE1.newick

'(r3:1,(r2:0.666667,(r1:0.333333,r0:0.333333)100:0.333333)100:0.333333);'

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te79d09b68b614d76b7a28a5ae0d98f7a"> r0 r1 r2 r3

In [5]:
test = hogtie.BinaryStateModel(TREE1,DATA1)
test.optimize()

03:06 | INFO    | optimize        | {'alpha': 0.899778, 'beta': 1.166458, 'Lik': -0.071025, 'negLogLik': 2.64, 'convergence': True}


In [7]:
test.tree.get_node_values('likelihood',True,True)

array([[0.49664593, 0.50335407],
       [0.38190178, 0.61809822],
       [0.08332826, 0.91667174],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ]])

In [9]:
test.log_lik

2.64

In [6]:
out = subprocess.run([
    "Rscript", 
    "-e", "library(ape)",
    "-e", f"tree <- read.tree(text='{TREE1.newick}')",
    "-e", f"data <- c{tuple(DATA1[::-1])}",
    "-e", "object <- ace(data, tree, marginal=TRUE, type='discrete', model='ARD', use.expm=TRUE)",
    "-e", "print(object)",
    "-e", "print(object$lik.anc)",
    ], capture_output=True,
)

print(out.stdout.decode())

NameError: name 'TREE1' is not defined

In [7]:
import os
import subprocess

In [8]:
HOGTIEDIR = os.path.dirname(os.getcwd())
HOGTIEDIR

'/Users/rachelcohen/hogtie/hogtie'

In [13]:
cmd = ["mkdir", f"{HOGTIEDIR}/hogtie_output"]
subprocess.run("mkdir", f"{HOGTIEDIR}/hogtieoutput)

CompletedProcess(args=['mkdir', '/Users/rachelcohen/hogtie/hogtie/hogtie_output'], returncode=0)

In [14]:
import pandas as pd

In [ ]:
pd.to_csv